In [46]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import datetime as datetime

dt = datetime.datetime.now()
np.random.seed(dt.microsecond)
tf.set_random_seed(dt.microsecond)

mnist = input_data.read_data_sets("/mnist/", one_hot=True)
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1], stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1+b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


num_filters2 = 64


W_conv2 = tf.Variable(tf.truncated_normal([5,5,num_filters1, num_filters2], stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2+b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

num_units1 = 7*7*num_filters2
num_units2 = 1024
h_pool2_flat = tf.reshape(h_pool2, [-1, num_units1])


w2 = tf.Variable(tf.truncated_normal([num_units1,num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2)+b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
k = tf.matmul(hidden2_drop, w0)+b0
p = tf.nn.softmax(k)

y_ = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=k,labels=y_)) 
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

for i in range(1, 1501):
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ts, keep_prob:0.8})

    if i % 500 == 0:
        loss_vals, acc_vals = [], []

        for c in range(4):
            start = int(len(mnist.test.labels) / 4 * c)
            end = int(len(mnist.test.labels) / 4 * (c+1))
            #print(start, end)
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:mnist.test.images[start:end], y_:mnist.test.labels[start:end], keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)

        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))



#saver.save(sess, 'cnn_session')
sess.close()


Extracting /mnist/train-images-idx3-ubyte.gz
Extracting /mnist/train-labels-idx1-ubyte.gz
Extracting /mnist/t10k-images-idx3-ubyte.gz
Extracting /mnist/t10k-labels-idx1-ubyte.gz
Step: 500, Loss: 0.776878, Accuracy: 0.948600
Step: 1000, Loss: 0.571646, Accuracy: 0.957300
Step: 1500, Loss: 0.449865, Accuracy: 0.965600


In [44]:
x33 = tf.constant([[1., 2., 3.],
                 [4., 5., 6.],
                 [7., 8., 9. ]])
x44 = tf.constant([[1., 2., 3., 4.],
                 [5., 6., 7., 8.],
                 [9., 10., 11., 12],
                 [13., 14., 15., 16]])

x = tf.reshape(x33, [1, 3, 3, 1])  # give a shape accepted by tf.nn.max_pool
x = tf.reshape(x44, [1, 4, 4, 1])
valid_pad = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
same_pad = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

valid_pad.get_shape()
same_pad.get_shape()
print(valid_pad, same_pad)
print(valid_pad.eval())
print(same_pad.eval())

Tensor("MaxPool_32:0", shape=(1, 2, 2, 1), dtype=float32) Tensor("MaxPool_33:0", shape=(1, 2, 2, 1), dtype=float32)
[[[[ 6.]
   [ 8.]]

  [[14.]
   [16.]]]]
[[[[ 6.]
   [ 8.]]

  [[14.]
   [16.]]]]
